In [ ]:
import sys
import subprocess

# 크롤링 및 모델링에 필요한 라이브러리 설치
packages = ["transformers", "beautifulsoup4", "requests", "lxml"] # lxml 추가 (XML 파싱용)
for package in packages:
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

from google.colab import drive
import os
import torch
import requests
import re
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Google Drive 마운트
drive.mount('/content/drive')

# 모델 경로 설정
PROJECT_DIR = "/content/drive/MyDrive/NLP_Assignment5"
MODEL_SAVE_DIR = os.path.join(PROJECT_DIR, "final_model")

print(f"🤖 학습된 모델 로드 중... ({MODEL_SAVE_DIR})")
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_DIR)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_DIR).to(device)
    print("✅ 모델 로드 성공! (Ready to Summarize)")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("Training Notebook을 먼저 실행하여 모델을 저장했는지 확인해주세요.")
    sys.exit()


# ==========================================
# 2. 크롤링 함수 정의 (블로그 본문 & RSS 파싱)
# ==========================================

def get_naver_blog_content(url):
    """
    네이버 블로그 URL -> 제목, 본문 추출 (Iframe 구조 대응)
    """
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        if "m.blog.naver.com" in url:
            url = url.replace("m.blog.naver.com", "blog.naver.com")

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        iframe = soup.select_one('iframe#mainFrame')
        if iframe:
            real_url = "https://blog.naver.com" + iframe['src']
            response = requests.get(real_url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')

        # 제목 추출
        title_elem = soup.select_one('.se-title-text') or soup.select_one('.htitle')
        title = title_elem.text.strip() if title_elem else "제목 없음"

        # 본문 추출
        content_elem = soup.select_one('.se-main-container') or soup.select_one('#postViewArea')

        if content_elem:
            text = content_elem.text
            text = re.sub(r'\n+', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            return title, text.strip()
        else:
            return None, None
    except Exception as e:
        print(f"크롤링 에러: {e}")
        return None, None

def get_latest_mofa_news(max_articles=5):
    """
    [자동화 핵심] 외교부 블로그 RSS를 뒤져서 '외교부 소식' 최신 글 URL을 가져옴
    지정된 개수만큼의 관련 게시글 URL과 제목을 리스트로 반환합니다.
    """
    # 외교부 블로그 ID: mofakr
    rss_url = "https://rss.blog.naver.com/mofakr.xml"
    articles_to_summarize = []

    try:
        response = requests.get(rss_url)
        # XML 파싱
        soup = BeautifulSoup(response.content, 'xml')

        items = soup.find_all('item')

        print(f"📡 외교부 블로그 RSS 검색 중... (총 {len(items)}개 최신 글 확인)")

        # 먼저 '외교부 소식' 관련 카테고리/제목 게시글을 수집
        for item in items:
            category = item.category.text if item.category else ""
            title = item.title.text
            link = item.link.text

            if "소식" in category or "보도" in category or "대변인" in category:
                articles_to_summarize.append((title, link))
                if len(articles_to_summarize) >= max_articles:
                    break

        # 만약 max_articles만큼 채워지지 않았다면, 최신 글부터 채워 넣음
        if len(articles_to_summarize) < max_articles:
            for item in items:
                title = item.title.text
                link = item.link.text
                # 중복 방지
                if (title, link) not in articles_to_summarize:
                    articles_to_summarize.append((title, link))
                    if len(articles_to_summarize) >= max_articles:
                        break

        if articles_to_summarize:
            print(f"✅ 총 {len(articles_to_summarize)}개의 타겟 게시글 발견!")
            for i, (title, link) in enumerate(articles_to_summarize):
                print(f"   - [{i+1}] {title}")
            return articles_to_summarize
        else:
            print("❌ '외교부 소식' 또는 유사한 키워드의 글을 찾지 못했습니다.")
            return []

    except Exception as e:
        print(f"RSS 파싱 실패: {e}")
        return []


# ==========================================
# 3. 요약 추론 함수
# ==========================================
def predict_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=150,
            min_length=30,
            length_penalty=1.5,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


# ==========================================
# 4. 실행: 외교부 소식 자동 요약기
# ==========================================
print("\n" + "="*60)
print("📢 [Daily 외교부 소식 자동 요약기]")
print("="*60)

# 1. RSS에서 최신 글 URL 자동 확보
target_articles = get_latest_mofa_news(max_articles=5) # 상위 5개 글 요약

if target_articles:
    for i, (article_title, target_url) in enumerate(target_articles):
        print(f"\n--- [게시글 {i+1}/{len(target_articles)}] ---")
        print(f"🔗 타겟 URL: {target_url}")
        print("⏳ 본문 크롤링 및 요약 수행 중...")

        # 2. 본문 가져오기
        title, content = get_naver_blog_content(target_url)

        if content:
            print(f"\n📄 [원문 정보]")
            print(f"제목: {title}")
            print(f"길이: {len(content)}자")

            # 3. 모델 요약
            summary = predict_summary(content)

            print("\n🤖 [AI 3줄 요약 결과]")
            print("-" * 40)
            print(summary)
            print("-" * 40)

        else:
            print(f"❌ 본문 추출 실패 (iframe 구조 변경 또는 접근 제한) - URL: {target_url}")
else:
    print("❌ 외교부 블로그에서 요약할 새 글을 찾지 못했습니다.")

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🤖 학습된 모델 로드 중... (/content/drive/MyDrive/NLP_Assignment5/final_model)
✅ 모델 로드 성공! (Ready to Summarize)

📢 [Daily 외교부 소식 자동 요약기]
📡 외교부 블로그 RSS 검색 중... (총 50개 최신 글 확인)
✅ 총 5개의 타겟 게시글 발견!
   - [1] 외교부 1차관, 美 국무부 정무차관 면담(12.2.)
   - [2] 제14차 한-국제원자력기구(IAEA) 고위급 정책협의회 개최
   - [3] 한국과 쿠바의 선율이 한 무대에, 한-쿠바 친선음악회 개최
   - [4] 민관합동 「수출‧수주 외교지원단」 출범 및 제1차 회의 개최
   - [5] 마약예방 수칙과 함께하는 안전한 해외여행

--- [게시글 1/5] ---
🔗 타겟 URL: https://blog.naver.com/mofakr/224096468765?fromRss=true&trackingCode=rss
⏳ 본문 크롤링 및 요약 수행 중...

📄 [원문 정보]
제목: 외교부 1차관, 美 국무부 정무차관 면담(12.2.)
길이: 826자

🤖 [AI 3줄 요약 결과]
----------------------------------------
박윤주 외교부 제1차관은 12.2.(화) 오후 미국 국무부에서 앨리슨 후커 국무부 정무차관을 면담하고  한일 정상회담 후속조치, ▴대북 공조 및 ▴지역·글로벌 정세 관련 의견을 교환하고 한미 관계 관련 오랜 전문성을 바탕으로 분야별 실무협의체 가동 등을 통해 구체적이고 가시적인 성과가 도출될 수 있도록 노력해달라고 당부했다.  ​ 박 차관은 두 차례의 성공적 한미 정상회담 개최로 한미 관계가 그 어느 때보다 굳건하게 발전하고 있다고 평가하고, 후